<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_05_Monitor_Reddit_mentions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a subReddit and monitor for company mentions.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [1]:
!pip install --upgrade graphlit-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [3]:
from typing import List, Optional

# Create entity extraction workflow, using Azure AI Text Analytics to identify organizations
async def create_workflow():
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Extraction",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.AZURE_COGNITIVE_SERVICES_TEXT,
                        extractedTypes=[enums.ObservableTypes.ORGANIZATION]
                    )
                )
            ]
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(name: str, workflow_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name=name,
        type=enums.FeedTypes.REDDIT,
        reddit=input_types.RedditFeedPropertiesInput(
            subredditName=name,
            readLimit=50 # limiting to 50 Reddit posts
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def query_organizations(name: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_organizations(
            filter=input_types.OrganizationFilter(
                name=name
            )
        )

        return response.organizations.results if response.organizations is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

# Locate contents where organization was observed (by Azure AI Text Analytics)
async def query_contents(organization_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                observations=[
                    input_types.ObservationReferenceFilter(
                        type=enums.ObservableTypes.ORGANIZATION,
                        observable=input_types.EntityReferenceFilter(
                            id=organization_id
                        )
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [4]:
from IPython.display import display, Markdown
import time

# NOTE: Fill in the Reddit subreddit name
subreddit_name = "Anthropic"

# NOTE: Fill in the organization name you're looking for
organization_name = "Google"

# Remove any existing feeds and workflows; only needed for notebook example
await delete_all_feeds()
await delete_all_workflows()

print('Deleted all feeds and workflows.')

workflow_id = await create_workflow()

if workflow_id is not None:
    print(f'Created workflow [{workflow_id}].')

    feed_id = await create_feed(name=subreddit_name, workflow_id=workflow_id)

    if feed_id is not None:
        print(f'Created feed [{feed_id}].')

        # Wait for feed to complete, since ingestion happens asychronously
        done = False
        time.sleep(5)
        while not done:
            done = await is_feed_done(feed_id)

            if not done:
                time.sleep(2)

        print(f'Completed feed [{feed_id}].')

        organizations = await query_organizations(organization_name)

        if organizations is not None and organizations.count != 0:
            for organization in organizations:
                if organization is not None:
                    print(f'Found organization [{organization.id}] named [{organization.name}].')

                    # Query contents by organization
                    contents = await query_contents(organization.id)

                    if contents is not None:
                        for content in contents:
                            if content is not None:
                                display(Markdown(f'### Found Reddit post [{content.id}] that mentioned [{organization_name}]'))
                                display(Markdown(content.markdown))
                                print()
        else:
            print(f'No organizations named [{organization_name}] found.')

Deleted all feeds and workflows.
Created workflow [1038c205-457a-4e87-83f5-51edb44df053].
Created feed [f1c586bb-2ff4-4af3-aaaf-4c4d3ec2d8e9].
Completed feed [f1c586bb-2ff4-4af3-aaaf-4c4d3ec2d8e9].
Found organization [947e908f-78e4-4801-a870-28d0687b594c] named [Google].


### Found Reddit post [93e46d7b-62d5-4431-a718-2e41c1874e58] that mentioned [Google]

Posted by u/mitt_brukernavn123 on Reddit at 8/11/2024 11:54:40 AM UTC: Would Anthropic win the race?
Hello! 

I'd love to hear your thoughts about this; 

There are many companies creating models. Why would Anthropic win over others? Personally I'd put money on it solely on Dario Amodel. He seems like a lovely person and I'd assume people would want to work with him (attracting the best talent, which is a strong indicator of success). 

The question I have is on his thought about the race to the top (incentivizing safe models through public opinion and eventually regulation) vs to the bottom (untethered AI). 

My thought is that putting limitations on the model will not win if the untethered AI performs better (although with the side effect of being able to help bad actors). Locally optimizing business owners wouldn't care (generalization) if the chosen model is "evil" if it gives him marginally better profits? And regulation is irrelevant (businesses take a calculated risk in any decision; how would it show on the bottom line?)

I support Dario's thinking but I can't see the clear and innate link to incentivizing business owners to buy Anthropic vs using Llama for instance. 

Have a great day, and sorry for any ignorance (just started exploring this and really like Anthropic)

u/BrutallyStupid commented at 8/11/2024 12:22:24 PM UTC:
I personally feel the models will be commoditised and heavily regulated and there will not be a single winner as such. And systems using the models will pick and choose the appropriate model (or combination of) for the task.

u/timberwolf007 commented at 8/11/2024 12:24:32 PM UTC:
It greatly depends on a couple of things. How educated people  generally are about the subject and how many people care about societal safety.  Most people are ignorant ( not stupid) of what A.I. is and what it’s capable of. And most people don’t care if the immediate outcome seems advantageous without considering how it could be used against them in the long term.

u/Gunner3210 commented at 8/11/2024 7:37:36 PM UTC:
The biggest risk is probably open source models making the race itself irrelevant.

Honestly, the race is already over. GPT-4o and 3.5 Sonnet are already much more capable than they need to be to power products and features that we can imagine right now.

eg: We have 100K+ context windows + perfect retrieval across the entire window. That's like 900 pages worth of text. 

As an LLM product innovator at the very forefront, I honestly don't know what to do with that, let alone more. The team I lead has built RAG, Knowledge graphs, metaprompting and agents, and we've applied these to products with a large enterprise userbase. 

We're rate-limited by product development, and customer adoption. Not reasoning abilities of models.

The power of products in connecting people to tech is formidable. Google and Meta have the upper hand here by a very large margin. Even smaller players like Slack for example, could roll something out tomorrow that truly delivers an autopilot experience, taking over menial work you don't want to do. Or work output generation tools like Figma or GitHub + Vscode etc automatically resolving Github issues. The list goes on and on.

I really doubt there is much of a future to organizations that solely train more capable models.

Yes, Anthropic has been able to move much faster than Meta, Google or even OpenAI. But they were small and nimble.

From experience, I can tell you that their talent acq is disorganized at best. Not saying this from a perspective of someone who got rejected. But rather as someone who jumped through the hoops, got perfect scores on their assessments and then got completely ghosted after.

So velocity wise, not sure if Anthropic will keep the lead for much longer. Strategically, it's likely that there is no more money in model training left. Products are where it's at, and neither OpenAI nor Anthropic are building any products that their own models cannot simply write develop in minutes with minor guidance from a single engineer.

u/Conscious-Map6957 commented at 8/11/2024 5:14:40 PM UTC:
Not with their non-existent customer support and payment issues. Not the mention all the random, non-explained banning I've heard from other users. Currently their application layer is very user-friendly and much better than ChatGPT's (talking about the artifacts and general UX), but that's not much of an advantage to depend on.

Despite some talent moving over to Anthropic, I don't see them making any leaps over OpenAI, Meta or Google. Ultimately I believe an open-source model with some polished UI and cute features will gain the most widespread adoption.



### Found Reddit post [75cf940d-cce5-4409-bf88-7034e848670b] that mentioned [Google]

Posted by u/AmirHotch on Reddit at 7/31/2024 12:37:20 PM UTC: Buyers aware-you might be charged monthly but cannot access your account.
I signed up a Claude account with google account, and subscribe the Claude Pro Plan which was charged monthly. Unfortunately, my Google account was disabled after the subscription. Due to this, I cannot log in to my Claude account, unable to use the Pro Plan benefits I'm paying for,and cannot cancel the subscription through normal means. My credit card is still being charged monthly for a service I cannot access.  
I have sent messages via every method I know. I delivery email to [support@anthropic.com](mailto:support@anthropic.com), but the mailbox responded automatically. I navigate to the help center and click the message icon, sent my issues to them, but no one responded me. I asked my account service, they said I had to contact the mechants.  
I still be charged, but can't cancel, can't utilize, can't contact Anthropic. I don't know what should I do now.  
Could anyone help me? I really need assistance, thank you.

Update: The team contacted me after I requested follow-up on discord, it has been 2 weeks since I submitted my messages. And my Google account finally restored. The issue has been solved. My gratitude to all of your sincere help and warmness.

u/dukhevych commented at 7/31/2024 12:48:00 PM UTC:
Funny thing - i got a response from anthropic when i wrote to them as a subscribed pro user. Before that all my messages were ignored. Probably someone could contact them mentioning your account. Share with me in inbox your email, I can try.

u/CapnWarhol commented at 7/31/2024 1:05:04 PM UTC:
Credit card block and chargeback

u/shadows_lord commented at 7/31/2024 1:49:52 PM UTC:
Anthropic really needs to get its shit together with all these bans. My work email got banned immediately after trying to get on the company's Team account. We moved to OpenAI Teams just because of this!

u/DaKuRa09 commented at 7/31/2024 1:45:36 PM UTC:
Maybe try to reporting them via discord? The admin there are very active



### Found Reddit post [3b2fb1ab-de3b-41cf-87b9-d761b134f570] that mentioned [Google]

Posted by u/Outside-Web-4118 on Reddit at 7/28/2024 7:44:00 PM UTC: Trying to test anthropic, but I can't free trial to start paying
A friend told me that if I'm tired of trying Open Ai, I can go and support Anthropic, but a problem arose when I wanted to register.

I registered normally, with my personal Google account, but when I went to verify my number, I got a message that I had already done it. This seemed strange to me, because I had never tried Anthropic. So I used another account so I could try it with another number that I have, but the same thing happened, then I asked my friend to help me, but the same thing happened again, and he used 3 different devices to register.  
  
I really want to support Anthropic, but not without trying it before giving them my money, I've been like this since mid-week, do you know if it's a mistake or there are no free trials anymore? If there isn't any more, why leave that sign up there?

u/cathline commented at 7/29/2024 9:34:56 PM UTC:
Mine said that my phone number was not qualified to sign up for the free credits.   

My carrier is google fi so maybe that's it, but it's not a temporary number.

u/Excellent-Try-9490 commented at 8/8/2024 11:16:16 AM UTC:
I had a lot of similar sign in issues with Open AI as well. I got tired of it and started using Poe.com which uses Claude by default. I use Poe primarily now and don’t use Open AI at all except for just comparing answers sometime. I find Claude Sonnet’s answers to be way more realistic, recent, detailed and relevant than OpenAI. So - they both have issues so we just have to deal with it. I finally got an OpenAI support person to contact me after several weeks. Maybe I just got lucky

